Load in datasets and add them to a graph. Add all triples to the knowledge graph as the these files reflect partitions from experiments in [COMET-ATOMIC 2020: On Symbolic and Neural Commonsense Knowledge Graphs](https://arxiv.org/pdf/2010.05953.pdf) (Hwang et al., 2021), however we are interested in the full ATOMIC 2020 knowledge base without partitions for now.

In [2]:
import csv
from collections import defaultdict

heads = defaultdict(list) # a dictionary with a head (string) as the key and a list of (relation, tail) tuples as the value (graph adjacency list)
tails = defaultdict(list) # same as above, but w/ tail as the key and a list of (relation, head) tuples as the value; all edge directions are thus reversed

train_file = open('data/train.tsv')
train = csv.reader(train_file, delimiter='\t')
for row in train:
    heads[row[0]].append((row[1], row[2]))
    tails[row[2]].append((row[1] + '⁻¹', row[0]))

dev_file = open('data/dev.tsv')
dev = csv.reader(dev_file, delimiter='\t')
for row in dev:
    heads[row[0]].append((row[1], row[2]))
    tails[row[2]].append((row[1] + '⁻¹', row[0]))

test_file = open('data/test.tsv')
test = csv.reader(test_file, delimiter='\t')
for row in dev:
    heads[row[0]].append((row[1], row[2]))
    tails[row[2]].append((row[1] + '⁻¹', row[0]))



We'll use a graph structure based on adjacency lists since I expect any knowledge base to be a relatively sparse graph (most nodes will only have edges to a small fraction of total nodes)

Some notes about the knowledge base: ATOMIC 2020 represents edges as unidirectional from the head to the tail. For our convencience, and because it could be useful for tasks, we give each node two adjacency lists. The heads adjacency list has edges in the direction that they are in the knowledge base, and the tails adjacency list has edges in the reversed direction for convenient accessing/lookups.

Though symmetric, reciprocal relationships are possible, this should not be as common as ATOMIC 2020 (unlike knowledge bases like ConceptNet) deliberately contains more information about cause-effect, if-then information. It is more focused on inferential knowledge than taxonomic knowledge. Let's print out a selection of five triplets from ATOMIC 2020 to show this.

In [3]:
import random

triples_list = list(heads.items())
len_triples = len(triples_list)
for i in range(5):
    random_index = random.randrange(len_triples)
    selection = triples_list[random_index]
    head = selection[0]
    relationship = selection[1][0][0]
    tail = selection[1][0][1]
    print(head + ' ' + relationship + ' ' + tail) # print the head and just one predicate (relationship + tail)


PersonX reaches PersonX's climax oEffect sigh and relax
PersonX is in high school oEffect receive gratitude for helping personx
yelp ObjectUse read reviews of the camps
karate gown ObjectUse move freely while learning
personx ObjectUse study together


For our shortest path function, we'll use a simple BFS search. There are no weights for ATOMIC 2020 allowing us to use this simple solution. 

If a more performant solution were needed, it would be interesting seeing if heuristics from using word embeddings + maybe a little depth would increase average performance. Although in the aforementioned paper about COMET-ATOMIC, (as far as I understand; need to ask) fast searches are not required as the knowledge bases are used to tune pretrained LM's. Computation time is therefore not noticeably increased when making a determination of a response to a question when the model is deployed.

In [4]:
from collections import deque

def find_shortest_path(starting_node, final_node):
    nodes_to_visit = deque([starting_node])
    paths_for_nodes = deque([starting_node]) # string with text of nodes previously visited
    nodes_visited = set() # for cycle detection
    nodes_to_visit_set = set() # need to not append nodes multiple times; otherwise runtime explodes
    while nodes_to_visit:
        current_node = nodes_to_visit[0]
        normal_edge_connections = heads[current_node]
        reversed_edge_connections = tails[current_node]
        connections_to_current_node = normal_edge_connections + reversed_edge_connections
        for connection in connections_to_current_node:
            relationship, connected_node = connection
            if connected_node == final_node:
                return paths_for_nodes[0] + ' ' + relationship + ' ' + connected_node # found final node
            elif connected_node not in nodes_visited and connected_node not in nodes_to_visit_set:
                nodes_to_visit.append(connected_node)
                nodes_to_visit_set.add(connected_node)
                paths_for_nodes.append(paths_for_nodes[0] + ' ' + relationship + ' ' + connected_node)
        nodes_to_visit.popleft()
        paths_for_nodes.popleft()
        nodes_visited.add(current_node)
    return 'No such path exists between ' + starting_node + ' and ' + final_node

# there is no path
print(find_shortest_path('PersonX washes everything', 'gibberish sfdjkhlghslkdfg'))
print(find_shortest_path('gibberish sfkjhafadf', 'gibberish sfdjkhlghslkdfg'))

# note that no weights with non-randomized BFS can some "catch-all" nodes getting high traffic; this may signify a more tenuous connection
# for both of these the person node plays a key factor in a short connection
print(find_shortest_path('happiness', 'go to dealership'))
print(find_shortest_path('cook curry', 'give up'))

# direct connection
print(find_shortest_path('dog', 'food'))

# some more examples
print(find_shortest_path('love', 'dog'))
print(find_shortest_path('PersonX washes everything', 'they ran out of soap'))
print(find_shortest_path('happiness', 'love'))

No such path exists between PersonX washes everything and gibberish sfdjkhlghslkdfg
No such path exists between gibberish sfkjhafadf and gibberish sfdjkhlghslkdfg
happiness NotDesires⁻¹ person Desires buy car xNeed go to dealership
cook curry HasSubEvent cook food CapableOf⁻¹ person NotDesires give up
dog Desires food
love ObjectUse⁻¹ dog
PersonX washes everything isAfter PersonX cleans the dishes HinderedBy they ran out of soap
happiness xIntent⁻¹ PersonX is having a great time xReact love
